In [1]:
%load_ext autoreload
%autoreload 2
#from gaia import *
import glob, os

import numpy as np
import IPython
from ipywidgets import *
import tkinter as tk
from tkinter import filedialog
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML
from itertools import chain
import random

from mpl_toolkits.mplot3d import Axes3D
import pickle
import re
import tensorflow as tf
import keras
print(keras.__version__)

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.utils import multi_gpu_model 
from keras.models import load_model


#%matplotlib notebook
#%matplotlib notebook 
%matplotlib inline
from matplotlib import ticker
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.tri as tri

import matplotlib.pyplot as plt
import matplotlib
font = {'family':'Times New Roman',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)
from matplotlib import rcParams
fontSize= 12
params = {'axes.labelsize': fontSize,'axes.titlesize':fontSize, 'font.size': fontSize, \
          'legend.fontsize': 12, 'xtick.labelsize':fontSize, 'ytick.labelsize': fontSize}
matplotlib.rcParams.update(params)
matplotlib.rc('text', usetex=True)

from decimal import Decimal
from sklearn import manifold

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

from joblib import load, dump, Parallel, delayed
import time

Using TensorFlow backend.


2.3.1


In [2]:
pathSave = "/plp_user/agar_si/gaia/python/Compressed_Forward_Mars/"

In [3]:
class getDataConvAE:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        for s in [0, 1, 2]:
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
                
            pathSaveE = pathSave + "processedDicts/Mars_new/ConvAE/" + sVar + "/train" 
            if not os.path.exists(pathSaveE):
                os.makedirs(pathSaveE)
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]
            
            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = [val[2]]
                else:
                    innertimeList.append(val[2])    
                    
            simList.append(dictInd)
            timeList.append(innertimeList)

            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            numBatches = len(simList) #int(np.ceil(totalSamples/self.batchSize)) 
            
            print(len(simList),len(timeList), totalSamples)  
                            
            def processEach(simInd):
                #print("------------" + str(batchN) + "----------------")
                if not os.path.exists(pathSaveE + "/" + str(simList[simInd])):
                    os.makedirs(pathSaveE + "/" + str(simList[simInd]))

                dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(simList[simInd]) + ".txt"
                with open(dictToOpen, "rb") as myFile:
                    profiles = load(myFile)
                paras = [np.log10(float(profiles['InputValues0'][1].split('=')[1])), \
                                  float(profiles['InputValues0'][5].split('=')[1]), \
                                  float(profiles['InputValues0'][6].split('=')[1]), \
                                  float(profiles['InputValues0'][7].split('=')[1]), \
                                  float(profiles['InputValues0'][8].split('=')[1].split("_")[0].split(".ini")[0])]
                with open(pathSaveE + "/" + str(simList[simInd]) + "/paras.txt", "wb") as f:
                    dump(paras , f, protocol=4)
                    
                #for t in timeList[simInd]:
                #    field = profiles['Dict_2D_' + sVar + '_time_' + str(t) + '_' + str(0)]
                #    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                    #field = (field-self.pMin)/(self.pMax-self.pMin)         
                    #with open(pathSaveE + "/" + str(simList[simInd]) + "/Dict_processed_data_MarsNew_2D_ConvAE_" + str(t) + ".txt", "wb") as f:
                    #    dump(field , f, protocol=4)

            Parallel(n_jobs=8, verbose=10)(delayed(processEach)(_bInd) for _bInd in range(len(simList)))
        
        profiles = {}

In [4]:
runThisCell = True
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['u', 'v', 'T']
    #batchSize = 100


    for sVar in sVariables:
        for n in numSims:
            dictToOpen = pathSave + "processedDicts/Mars_new/" + sVar + "/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.9trPercent.txt"
            with open(dictToOpen, "rb") as myFile:
                indexerDict = load(myFile)
            Dict_processed_data = {}
            data = getDataConvAE(n, pathSave, numChannels, sVar, indexerDict)
            #Dict_processed_data['x'] = indexerDict["x"] 
            #Dict_processed_data['y'] = indexerDict["y"] 
            #Dict_processed_data['pMax'] = data.pMax
            #Dict_processed_data['pMin'] = data.pMin

            #with open(pathSave + "processedDicts/Mars_new/ConvAE/" + sVar + "1/Dict_processed_data_MarsNew_2D_ConvAE_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel.txt", "wb") as f:
            #    dump(Dict_processed_data, f, protocol=4)

9080 9080 958370


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   10.4s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   17.1s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   24.1s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   28.5s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   29.3s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   30.0s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   30.5s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   41.0s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   55.7s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  3.2min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  3.7min
[Parallel(

501 501 51174


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    7.8s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   14.5s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   21.4s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   27.6s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   38.3s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   46.2s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   57.0s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  3.0min
[Parallel(n_jobs=8)]: Do

499 499 50835


...

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   12.3s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   19.1s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   27.6s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   38.2s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   46.3s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   58.9s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  1.4min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  2.7min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  3.0min
[Parallel(

501 501 51174


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    8.1s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   15.6s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   29.6s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   42.3s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   55.5s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  2.6min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  4.0min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  4.5min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  5.2min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  5.8min
[Parallel(n_jobs=8)]: Do

499 499 50835


[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   10.0s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:   16.6s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:   24.3s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   33.1s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:   44.4s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:   53.0s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  2.8min
[Parallel(n_jobs=8)]: Done 205 tasks      | elapsed:  3.1min
[Parallel(n_jobs=8)]: Do

In [5]:
class getDataConvAE_3D:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
                
            pathSaveE = pathSave + "processedDicts/Mars_new/ConvAE/" + sVar + "/" + stringInt
            if not os.path.exists(pathSaveE):
                os.makedirs(pathSaveE)
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]
            
            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = [val[2]]
                else:
                    innertimeList.append(val[2])    
                    
            simList.append(dictInd)
            timeList.append(innertimeList)
            
            indexerVec = []
            tol = 1e-3
            
            numSims = len(simList)
            maxTimeSteps = 45
            spacedTimeVec = np.linspace(0,0.04913813148788927,maxTimeSteps)
            
            for ind, sim in enumerate(simList):
                innerTimeList = []
                for time in spacedTimeVec:
                    whereVec = np.where((time>=np.asarray(timeList[ind])-tol) & (time<=np.asarray(timeList[ind])+tol))[0]
                    if len(whereVec) > 0:
                        indexerVec.append([sim,0,timeList[ind][whereVec[-1]]])
                    else:
                        indexerVec.append([sim,0,-1])
            
            print(numSims)
            
            self.x_data = np.zeros((numSims,5))
            self.y_data = np.zeros((numSims,maxTimeSteps,302,394,1))  
            
            print("memory allocated")

            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            numBatches = len(simList) #int(np.ceil(totalSamples/self.batchSize)) 
                                        
            dictPrev = -1
            dictInd = -1            
            for ind, i in enumerate(indexerVec):
                if i[0] != dictPrev:
                    dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i[0]) + ".txt"
                    with open(dictToOpen, "rb") as myFile:
                        profiles = load(myFile)
                    
                   
                    dictPrev = i[0]
                    timeStep = 0
                    dictInd += 1
                    
                    self.x_data[dictInd,:] = [np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]

                if i[2] != -1:
                    field = profiles['Dict_2D_' + sVar + '_time_' + str(i[2]) + '_' + str(i[1])]
                    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                    #field = (field-self.oMin)/(self.oMax-self.oMin) 
                    self.y_data[dictInd,timeStep,:,:,:] = field #encoder.predict(field)
                
                timeStep += 1
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:],axis=0)
                paraMin = np.amin(m[:,:],axis=0)
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m        
            
            self.x_data = normalizeMatrix(self.x_data)
            
            print(self.xMax, self.xMin) #, self.deltatMax, self.deltatMin)
            
            plt.figure()
            plt.plot(self.y_data[0:10,:,:,:,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.x_data[0:10,:].flatten(),'r.')
            plt.show()
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            #self.y_data = self.y_data[indices,:,:,:,:]
            #self.x_data = self.x_data[indices,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_data.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_data.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_cv.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_cv.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 2:
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_test.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvAE/" + sVar + "/Dict_processed_data_MarsNew_3DConv_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_test.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
        
        profiles = {}

In [6]:
runThisCell = False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'u', 'v'] #, 'T']
    #batchSize = 100

    for sVar in sVariables:
        for n in numSims:
            dictToOpen = pathSave + "processedDicts/Mars_new/" + sVar + "/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.98trPercent.txt"
            with open(dictToOpen, "rb") as myFile:
                indexerDict = load(myFile)
            Dict_processed_data = {}
            data = getDataConvAE_3D(n, pathSave, numChannels, sVar, indexerDict)
            #Dict_processed_data['x'] = indexerDict["x"] 
            #Dict_processed_data['y'] = indexerDict["y"] 
            #Dict_processed_data['pMax'] = data.pMax
            #Dict_processed_data['pMin'] = data.pMin

            #with open(pathSave + "processedDicts/Mars_new/ConvAE/" + sVar + "1/Dict_processed_data_MarsNew_2D_ConvAE_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel.txt", "wb") as f:
            #    dump(Dict_processed_data, f, protocol=4)

In [7]:
class getDataNN_POD:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f7A11s3_c369', "conv2d_3", 1620, [12,15,9], -7] 
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)
        
        dictToOpen = "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str([12000]) + "sims_0.98trPercent.txt"
        with open(self.pathSave + dictToOpen, "rb") as myFile:
            Dict_processed_data_monly = load(myFile)
        
        self.oMax = Dict_processed_data_monly['pMax']
        self.oMin = Dict_processed_data_monly['pMin']
        
        def POD(X,r):
            U,Sigma,VT = np.linalg.svd(X,full_matrices=0) # Step 1
            Ur = U[:,:r]
            Sigmar = np.diag(Sigma[:r])
            VTr = VT[:r,:]
            return Ur, Sigmar, VTr
        
        self.Ur_min = 1e+16
        self.Sigmar_min = 1e+16        
        self.VTr_min = 1e+16

        self.Ur_max = -1e+16
        self.Sigmar_max = -1e+16
        self.VTr_max = -1e+16
        
        numModes = 100

        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
                
            pathSaveE = pathSave + "processedDicts/Mars_new/NN/" + sVar + "/" + stringInt
            if not os.path.exists(pathSaveE):
                os.makedirs(pathSaveE)
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]
            
            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = [val[2]]
                else:
                    innertimeList.append(val[2])    
                    
            simList.append(dictInd)
            timeList.append(innertimeList)
            
            indexerVec = []
            tol = 1e-3
            
            numSims = len(simList)
            maxTimeSteps = numModes
            spacedTimeVec = np.linspace(0,0.04913813148788927,maxTimeSteps)
            spaced_timeList = []
            
            for ind, sim in enumerate(simList):
                innerTimeList = []
                for time in spacedTimeVec:
                    whereVec = np.where((time>=np.asarray(timeList[ind])-tol) & (time<=np.asarray(timeList[ind])+tol))[0]
                    if len(whereVec) > 0:
                        innerTimeList.append(timeList[ind][whereVec[-1]])
                    else:
                        innerTimeList.append(-1)
                spaced_timeList.append(innerTimeList)
            
            print(numSims)            
            self.x_data = np.zeros((numSims*numModes,6))
            self.y_data = np.zeros((numSims*numModes,ae[2]+1+numModes))  

            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            numBatches = len(simList) #int(np.ceil(totalSamples/self.batchSize)) 
                                        
            dictPrev = -1
            dictInd = -1            
            for ind, i in enumerate(simList):
                if ind%10==0:
                    print(ind)
                dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i) + ".txt"
                with open(dictToOpen, "rb") as myFile:
                    profiles = load(myFile)
                    
                X = np.zeros((ae[2],maxTimeSteps))
                for ind_t,t in enumerate(spaced_timeList[ind]):
                    if t != -1:
                        field = profiles['Dict_2D_' + sVar + '_time_' + str(t) + '_' + str(0)]
                        field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                        field = (field-self.oMin)/(self.oMax-self.oMin)                     
                        X[:,ind_t] = encoder.predict(field).flatten()
                
                Ur, Sigmar, VTr = POD(X,numModes)
                
                for m in range(numModes):
                    Sigmar[m,m] = np.log10(Sigmar[m,m])

                if s==0:
                    if np.amin(Ur) < self.Ur_min:
                        self.Ur_min = np.amin(Ur)
                    if np.amin(Sigmar) < self.Sigmar_min:
                        self.Sigmar_min = np.amin(Sigmar)
                    if np.amin(VTr) < self.VTr_min:
                        self.VTr_min = np.amin(VTr)

                    if np.amax(Ur) > self.Ur_max:
                        self.Ur_max = np.amax(Ur)
                    if np.amax(Sigmar) > self.Sigmar_max:
                        self.Sigmar_max = np.amax(Sigmar)
                    if np.amax(VTr) > self.VTr_max:
                        self.VTr_max = np.amax(VTr)
                
                for m in range(numModes):
                    self.y_data[(ind*numModes)+m,:ae[2]] = Ur[:,m]
                    self.y_data[(ind*numModes)+m,ae[2]:ae[2]+1] = Sigmar[m,m]
                    self.y_data[(ind*numModes)+m,ae[2]+1:] = VTr[m,:]
                    
                    if spaced_timeList[ind][m] == -1:
                        mode = -1
                    else:
                        mode = m
                        
                    self.x_data[(ind*numModes)+m,:] = [mode,\
                       np.log10(float(profiles['InputValues'+ str(0)][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(0)][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(0)][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(0)][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(0)][8].split('=')[1].split("_")[0].split(".ini")[0])]
                    
            print(self.x_data.shape, self.y_data.shape)
            
            discard_indices = []
            for ind in range(self.y_data.shape[0]):
                mode = int(self.x_data[ind,0])
                if mode != -1:
                    self.y_data[ind,:ae[2]] = (self.y_data[ind,:ae[2]]-self.Ur_min)/(self.Ur_max-self.Ur_min)
                    self.y_data[ind,ae[2]:ae[2]+1] = (self.y_data[ind,ae[2]:ae[2]+1]-self.Sigmar_min)/(self.Sigmar_max-self.Sigmar_min)
                    self.y_data[ind,ae[2]+1:] = (self.y_data[ind,ae[2]+1:]-self.VTr_min)/(self.VTr_max-self.VTr_min)
                else:
                    discard_indices.append(ind)
                    
            self.y_data = np.delete(self.y_data, discard_indices, axis=0)
            self.x_data = np.delete(self.x_data, discard_indices, axis=0)
            
            print(self.x_data.shape, self.y_data.shape)  

            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:],axis=0)
                paraMin = np.amin(m[:,:],axis=0)
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m        
            
            self.x_data = normalizeMatrix(self.x_data) 
            
            print(self.xMax, self.xMin) 
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.y_data = self.y_data[indices,:]
            self.x_data = self.x_data[indices,:]
            
            plt.figure()
            plt.plot(self.y_data[0:1000,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.x_data[0:1000,:].flatten(),'r.')
            plt.show()
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_data.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_data.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_cv.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_cv.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 2:
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_regIndexerVec_test.txt", "wb") as f:
                    dump(indexerVec , f, protocol=4)
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_NNpod_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_test.txt", "wb") as f:
                    dump([self.x_data,self.y_data, self.xMax, self.xMin, self.Ur_min, self.Sigmar_min, self.VTr_min, self.Ur_max, self.Sigmar_max, self.VTr_max] , f, protocol=4)
        
        profiles = {}

In [8]:
runThisCell = False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'u', 'v'] #, 'T']
    #batchSize = 100

    for sVar in sVariables:
        for n in numSims:
            dictToOpen = pathSave + "processedDicts/Mars_new/" + sVar + "/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.98trPercent.txt"
            with open(dictToOpen, "rb") as myFile:
                indexerDict = load(myFile)
            Dict_processed_data = {}
            data = getDataNN_POD(n, pathSave, numChannels, sVar, indexerDict)

In [9]:
class getDataConvNN_wiMelt:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f7A11s3_c369_tanh_l2reg_wlrsc', "conv2d_3", 1620, [12,15,9], -7] 
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)
        
       
        dictToOpen = "/plp_user/agar_si/gaia/python/Data/1D/Compare_melt/savedDict_profiles_Mars_new_1D_Dict" + str(0) + ".txt"
        with open(dictToOpen, "rb") as myFile:
            profiles = load(myFile)
        
        self.rProf = profiles['Dict_Rprof_4p5Gyr' + str(0)]
        self.rProf = (self.rProf - self.rProf.min())/(self.rProf.max()-self.rProf.min())
        
        self.oMax = indexerDict['pMax']
        self.oMin = indexerDict['pMin']
        
        totalTime = 0.04913813148788927
        H_U_238  = 9.46 * 1e-5 
        H_U_235 = 5.69 * 1e-4 
        H_Th_232 = 2.64 * 1e-5 
        H_K_40 = 2.92 * 1e-5 
        half_life_U_238 = 4.47 * 1e9 
        half_life_U_235 = 7.04 * 1e8 
        half_life_Th_232 = 1.40 * 1e10 
        half_life_K_40 = 1.25 * 1e9 
        year_to_seconds = 365.25 * 24. * 3600.
        D = 1700e+3
        rho_c = 7000. 
        rho_m   = 3500. 
        c_p_c = 850. 
        c_p_m = 1200. 
        r_m = 1700e+3
        r_c = 1700e+3
        alpha_m = 2.5e-5
        alpha_c = 4.0e-5 
        
        T_delta = 2000
        f2 = 1./(c_p_m)
        f0 = 3./(rho_m*c_p_m) * np.power(r_c,2)/(np.power(r_c+r_m,3)-np.power(r_c,3))
        f1 = 3./(rho_m*c_p_m) * np.power(r_c+r_m,2)/(np.power(r_c+r_m,3)-np.power(r_c,3))
        d_0 = 2.47
        d_1 = 0.33e-9
        d_2 = 0.48
        k_ref = 4
        
        k_diffusive = k_ref/(rho_m*c_p_m)        #1e-6 
        g = 3.7
        lambda_U_238 = np.log(2.)/(half_life_U_238 * year_to_seconds / np.power(D,2.) * k_diffusive)
        lambda_U_235 = np.log(2.)/(half_life_U_235 * year_to_seconds / np.power(D,2.) * k_diffusive)
        lambda_Th_232 = np.log(2.)/(half_life_Th_232 * year_to_seconds / np.power(D,2.) * k_diffusive)
        lambda_K_40 = np.log(2.)/(half_life_K_40 * year_to_seconds / np.power(D,2.) * k_diffusive)

        r_c = 1700e+3
        r_p = 3400e+3
        D = 1700e+3 
        rho_crust = 2900.
        crustal_thickness = 64.3e+3
        r_crust = r_p - crustal_thickness
        
        m_cr = (np.power(r_p,3.) - np.power(r_crust,3.)) * rho_crust * 4/3 * np.pi
        m_m = (np.power(r_crust,3.) - np.power(r_c,3.)) * rho_m * 4/3 * np.pi

        def getH0(_t, depleteBy):
            C_U_0 = 16.0 * 1e-9 * depleteBy
            C_Th_0_232 = 56.0 * 1e-9 * depleteBy
            C_K_0 = 305.0 * 1e-6 * depleteBy
            _t = ((_t/k_diffusive) * np.power(D,2.))/year_to_seconds
            H_t_U_238 = 0.9928 * C_U_0 * H_U_238  * np.exp(_t * np.log(2.) / half_life_U_238) 
            H_t_U_235 = 0.0072 * C_U_0 * H_U_235 * np.exp(_t * np.log(2.) / half_life_U_235)
            H_t_Th_232 = C_Th_0_232 * H_Th_232 * np.exp(_t * np.log(2.) / half_life_Th_232) 
            H_t_K_40 = 1.19 * 1e-4 * C_K_0 * H_K_40 * np.exp(_t * np.log(2.) / half_life_K_40)
            return H_t_U_238 + H_t_U_235 + H_t_Th_232 + H_t_K_40  

        Tcmb = 250+T_delta
        
        def getParaT(Lambda_cr,tprof,t_current,t_prev):
            tprof.shape = (302,)
            tprof = tprof*T_delta+250
            depleteBy = m_m/(m_cr*Lambda_cr + m_m)
            deltaTimeD = (t_current-t_prev)/k_diffusive*np.power(D,2.)
            k_top = (d_0 + d_1*(rho_m*g*0))*np.power(300/(tprof[-1]),d_2)
            k_bot = (d_0 + d_1*(rho_m*g*D))*np.power(300/(tprof[0]),d_2)
            qbot = -((tprof[0]-tprof[1]))/((self.rProf[0]-self.rProf[1])*D) * k_bot 
            qtop = -((tprof[-2]-tprof[-1]))/((self.rProf[-2]-self.rProf[-1])*D) * k_top
            T_integ  = np.mean(tprof) + ((f0*qbot - f1*qtop + f2*getH0(totalTime-t_current,depleteBy)) * deltaTimeD)
            return T_integ
        
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = []
                else:
                    innertimeList.append(val[2])
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = 0
            for iS in range(len(simList)):
                totalSamples += len(timeList[iS])-1
            print(totalSamples)
            self.y_data = np.zeros((totalSamples,2+ae[3][0]*ae[3][1]*ae[3][2]))
            self.x_data = np.zeros((totalSamples,1+6))
                    
            dictPrev = -1
            ind = 0
            for indSim, valSim in enumerate(simList):  
                dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(valSim) + ".txt"
                with open(dictToOpen, "rb") as myFile:
                    profiles = load(myFile)

                for indTime, valTime in enumerate(timeList[indSim][:-1]):  
                    t_current = valTime
                    t_next = timeList[indSim][indTime+1]
                    
                    field = profiles['Dict_2D_' + sVar + '_time_' + str(t_current) + '_' + str(0)]
                    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)
                    tProf = np.mean(field,axis=2,keepdims=False)
                    
                    field_next = profiles['Dict_2D_' + sVar + '_time_' + str(t_next) + '_' + str(0)]
                    field_next.shape = (1,numLayers,sizeEachLayer,numStateVariables)
                    tProf_next = np.mean(field_next)
                    T_next = tProf_next*2000+250
                    
                    Lambda_cr = float(profiles['InputValues'+ str(0)][7].split('=')[1])
                    meltedTemp = getParaT(Lambda_cr,tProf,t_next,t_current) - T_next
                    self.y_data[ind,0] = T_next
                    self.y_data[ind,1] = meltedTemp
                    
                    field = (field-self.oMin)/(self.oMax-self.oMin) 
                    self.y_data[ind,2:] = encoder.predict(field).flatten()

                    self.x_data[ind,:] = [float(t_next), float(t_current), \
                            np.log10(float(profiles['InputValues'+ str(0)][1].split('=')[1])), \
                                    float(profiles['InputValues'+ str(0)][5].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][6].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][7].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][8].split('=')[1].split("_")[0].split(".ini")[0])]
                    ind += 1
                    
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,0:7],axis=0)
                paraMin = np.amin(m[:,0:7],axis=0)
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0 and ind<7:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m
            
            self.x_data = normalizeMatrix(self.x_data)
            
            #if s==0:
            #    self.yMax = [np.amax(self.y_data[:,0]), np.amax(self.y_data[:,1]), np.amax(self.y_data[:,2:])]
            #    self.yMin = [np.amin(self.y_data[:,0]), np.amin(self.y_data[:,1]), np.amin(self.y_data[:,2:])]

            #self.y_data[:,0] = (self.y_data[:,0]-self.yMin[0])/(self.yMax[0]-self.yMin[0]) 
            self.y_data[:,1] = (self.y_data[:,1]-250)/2000 
            self.y_data[:,0] = (self.y_data[:,0]-250)/2000
            
            print(self.xMax, self.xMin) #, self.yMax, self.yMin)
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:]
            self.y_data = self.y_data[indices,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_wMelt_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae1620_x_data.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_wMelt_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae1620_x_cv.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_wMelt_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae1620_x_test.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [10]:
runThisCell=False
if runThisCell:
    numSims = [10000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.9trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataConvNN_wiMelt(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['xMax'] = data.xMax
            Dict_processed_data['xMin'] = data.xMin
            #Dict_processed_data['yMax'] = data.yMax
            #Dict_processed_data['yMin'] = data.yMin
            Dict_processed_data['oMax'] = data.oMax
            Dict_processed_data['oMin'] = data.oMin
            Dict_processed_data['rProf'] = data.rProf

            with open(pathSave+ "processedDicts/Mars_new/NN/T/Dict_processed_data_MarsNew_2D_NN_wMelt_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae1620.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)

In [11]:
class getDataConvNN:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict, ae):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        dictToOpen = "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str([12000]) + "sims_0.9trPercent.txt"
        with open(self.pathSave + dictToOpen, "rb") as myFile:
            Dict_processed_data_monly = load(myFile)
        
        self.oMax = Dict_processed_data_monly['pMax']
        self.oMin = Dict_processed_data_monly['pMin']
        
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)
        
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
                    
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = [val[2]]
                else:
                    innertimeList.append(val[2])    
                    
            simList.append(dictInd)
            timeList.append(innertimeList)
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = 0
            for iS in range(len(simList)):
                totalSamples += len(timeList[iS])
            
            self.y_data = np.zeros((totalSamples,ae[3][0],ae[3][1],ae[3][2]))
            self.x_data = np.zeros((totalSamples,6))
                    
            ind = 0
            for indSim, valSim in enumerate(simList):  
                print(indSim)
                dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(valSim) + ".txt"
                with open(dictToOpen, "rb") as myFile:
                    profiles = load(myFile)

                for indTime, valTime in enumerate(timeList[indSim]):  
                    t_current = valTime
                    
                    field = profiles['Dict_2D_' + sVar + '_time_' + str(t_current) + '_' + str(0)]
                    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)
                    
                    field = (field-self.oMin)/(self.oMax-self.oMin) 
                    self.y_data[ind,:] = encoder.predict(field)

                    self.x_data[ind,:] = [float(t_current), \
                            np.log10(float(profiles['InputValues'+ str(0)][1].split('=')[1])), \
                                    float(profiles['InputValues'+ str(0)][5].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][6].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][7].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][8].split('=')[1].split("_")[0].split(".ini")[0])]
                    ind += 1
                    
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,0:6],axis=0)
                paraMin = np.amin(m[:,0:6],axis=0)
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0 and ind<6:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m
            
            self.x_data = normalizeMatrix(self.x_data)
            
            if s==0:
                self.yMax = np.amax(self.y_data)
                self.yMin = np.amin(self.y_data)
                
            self.y_data = (self.y_data-self.yMin)/(self.yMax-self.yMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin)
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:]
            self.y_data = self.y_data[indices,:,:,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae" + str(ae[2]) + "_x_data_0p98tr.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae" + str(ae[2]) + "_x_cv_0p98tr.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/NN/" + sVar + "/Dict_processed_data_MarsNew_2D_NN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae" + str(ae[2]) + "_x_test_0p98tr.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [12]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']
    
    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_[12000]sims_0.98trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)
        
    aeList = [['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11]]
    
    for ae in aeList:
        for sVar in sVariables:
            for n in numSims:
                Dict_processed_data = {}
                data = getDataConvNN(n, pathSave, numChannels, sVar, indexerDict, ae)
                Dict_processed_data['x'] = data.x 
                Dict_processed_data['y'] = data.y 
                Dict_processed_data['xMax'] = data.xMax
                Dict_processed_data['xMin'] = data.xMin
                Dict_processed_data['yMax'] = data.yMax
                Dict_processed_data['yMin'] = data.yMin

                with open(pathSave+ "processedDicts/Mars_new/NN/T/Dict_processed_data_MarsNew_2D_NN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae" + str(ae[2]) + "_0p98tr.txt", "wb") as f:
                    dump(Dict_processed_data, f, protocol=4)

In [13]:
class getDataConvLSTM:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11]   
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)   
        
        dictToOpen = "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str([12000]) + "sims_0.9trPercent.txt"
        with open(self.pathSave + dictToOpen, "rb") as myFile:
            Dict_processed_data_monly = load(myFile)
        
        self.oMax = Dict_processed_data_monly['pMax']
        self.oMin = Dict_processed_data_monly['pMin']
        
        
        maxTimeSteps = 0
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                    if timeSteps > maxTimeSteps:
                        maxTimeSteps = timeSteps
        
        for s in range(1,3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                
            print(numSims,maxTimeSteps)
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            
            self.x_data = np.zeros((numSims,maxTimeSteps,ae[3][0],ae[3][1],ae[3][2]+1))
                    
            dictPrev = -1
            dictInd = -1
            for ind, i in enumerate(indexerVec):
                
                if i[0] != dictPrev:
                    dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i[0]) + ".txt"
                    with open(dictToOpen, "rb") as myFile:
                        profiles = load(myFile)
                    dictPrev = i[0]
                    timeStep = 0
                    dictInd += 1

                    print(dictInd,i)
                
                field = profiles['Dict_2D_' + sVar + '_time_' + str(i[2]) + '_' + str(i[1])]
                field.shape = (1,numLayers,sizeEachLayer,numStateVariables)
                
                field = (field-self.oMin)/(self.oMax-self.oMin) 
                self.x_data[dictInd,timeStep,:,:,1:] = encoder.predict(field)
                
                if ind != len(indexerVec)-1 and indexerVec[ind][0] == indexerVec[ind+1][0]:
                    t_delta = float(indexerVec[ind+1][2])-float(i[2])
                    mask = 1
                else:
                    t_delta = 0
                    mask = 0
                    self.x_data[dictInd,timeStep+1:,0,0:7,0] = [mask, t_delta, \
                        np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]

                self.x_data[dictInd,timeStep,1,0,0] = float(i[2])
                self.x_data[dictInd,timeStep,0,0:7,0] = [mask, t_delta, \
                        np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]
                timeStep += 1
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:,0,0:7,0],axis=(0,1)).tolist()
                paraMin = np.amin(m[:,:,0,0:7,0],axis=(0,1)).tolist()
                
                paraMax.insert(0,np.amax(self.x_data[:,:,1,0,0]))
                paraMin.insert(0,np.amin(self.x_data[:,:,1,0,0]))
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = np.asarray(paraMax)-np.asarray(paraMin)
                for ind,val in enumerate(paraNorms[1:]):
                    if val > 0:
                        m[:,:,0,ind,0] = np.divide(m[:,:,0,ind,0]-paraMin[ind+1],val)
                        
                m[:,:,1,0,0] = (m[:,:,1,0,0]-paraMin[0])/paraNorms[0]
                return m        
            
            self.x_data = normalizeMatrix(self.x_data)
            
            if s==0:
                self.yMax = np.amax(self.x_data[:,:,:,:,1:])
                self.yMin = np.amin(self.x_data[:,:,:,:,1:])
            
            self.x_data[:,:,:,:,1:] = (self.x_data[:,:,:,:,1:]-self.yMin)/(self.yMax-self.yMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin)
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,0,:,0].flatten(),'b.')
            plt.show()
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,1,0,0].flatten(),'g.')
            plt.show()
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,:,:,1:].flatten(),'r.')
            plt.show()
            
            if s<2:
                indices = np.arange(self.x_data.shape[0])       
                random.seed(6)
                random.shuffle(indices)
                self.x_data = self.x_data[indices,:,:,:,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvLSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_ConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_data_0p98tr_wt.txt", "wb") as f:
                    dump(self.x_data , f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvLSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_ConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_cv_0p98tr_wt.txt", "wb") as f:
                    dump(self.x_data, f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/ConvLSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_ConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_test_0p98tr_wt.txt", "wb") as f:
                    dump(self.x_data, f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [14]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.98trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataConvLSTM(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['paraMax'] = data.xMax
            Dict_processed_data['paraMin'] = data.xMin
            Dict_processed_data['postEncodedOMax'] = data.yMax
            Dict_processed_data['postEncodedOMin'] = data.yMin

            with open(pathSave+ "processedDicts/Mars_new/ConvLSTM/T/Dict_processed_data_MarsNew_2D_ConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_0p98tr_wt.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)

In [15]:
class getDataLSTM:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11] 
        #ae = ['T_Mars300_sims10k_f7A11s3_c369', "conv2d_3", 1620, [12,15,9], -7] 
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)   
        
        dictToOpen = "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str([12000]) + "sims_0.9trPercent.txt"
        with open(self.pathSave + dictToOpen, "rb") as myFile:
            Dict_processed_data_monly = load(myFile)
        
        self.oMax = Dict_processed_data_monly['pMax']
        self.oMin = Dict_processed_data_monly['pMin']
        
        
        maxTimeSteps = 0
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                    if timeSteps > maxTimeSteps:
                        maxTimeSteps = timeSteps
        
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                
            print(numSims,maxTimeSteps)
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            
            self.y_data = np.zeros((numSims,maxTimeSteps,ae[3][0]*ae[3][1]*ae[3][2]))
            self.x_data = np.zeros((numSims,maxTimeSteps,ae[3][0]*ae[3][1]*ae[3][2]+8))
                    
            dictPrev = -1
            dictInd = -1
            for ind, i in enumerate(indexerVec):
                if ind%1000==0:
                    print(str(ind/len(indexerVec)*100))
                if i[0] != dictPrev:
                    dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i[0]) + ".txt"
                    with open(dictToOpen, "rb") as myFile:
                        profiles = load(myFile)
                    dictPrev = i[0]
                    timeStep = 0
                    dictInd += 1
                    
                field = profiles['Dict_2D_' + sVar + '_time_' + str(i[2]) + '_' + str(i[1])]
                field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                field = (field-self.oMin)/(self.oMax-self.oMin) 
                self.x_data[dictInd,timeStep,8:] = encoder.predict(field).flatten()
                
                if ind != len(indexerVec)-1 and indexerVec[ind][0] == indexerVec[ind+1][0]:
                    t_delta = float(indexerVec[ind+1][2])-float(i[2])
                    t_ = float(i[2])
                    field = profiles['Dict_2D_' + sVar + '_time_' + str(indexerVec[ind+1][2]) + '_' + str(indexerVec[ind+1][1])]
                    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                    field = (field-self.oMin)/(self.oMax-self.oMin) 
                    self.y_data[dictInd,timeStep,:] = encoder.predict(field).flatten()
                    mask = 1
                else:
                    t_delta = 0
                    mask = 0
                    t_ = 0
                    self.x_data[dictInd,timeStep+1:,0:8] = [mask, t_delta, t_,\
                        np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]

                self.x_data[dictInd,timeStep,0:8] = [mask, t_delta, t_, \
                        np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]
                timeStep += 1
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:,0:8],axis=(0,1))
                paraMin = np.amin(m[:,:,0:8],axis=(0,1))
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,:,ind] = np.divide(m[:,:,ind]-paraMin[ind],val)
                return m        
            
            self.x_data = normalizeMatrix(self.x_data)
            
            if s==0:
                self.yMax = np.amax(self.y_data)
                self.yMin = np.amin(self.y_data)

            #print("NOT normalizing encoded representation between 0 and 1")
            self.y_data = (self.y_data-self.yMin)/(self.yMax-self.yMin) 
            self.x_data[:,:,8:] = (self.x_data[:,:,8:]-self.yMin)/(self.yMax-self.yMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin)
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.y_data[0:100,:,:].flatten(),'r.')
            plt.show()
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:,:]
            self.y_data = self.y_data[indices,:,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_data_ae840.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_cv_ae840.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_test_ae840.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [16]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.98trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataLSTM(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['paraMax'] = data.xMax
            Dict_processed_data['paraMin'] = data.xMin
            Dict_processed_data['postEncodedOMax'] = data.yMax
            Dict_processed_data['postEncodedOMin'] = data.yMin

            with open(pathSave+ "processedDicts/Mars_new/LSTM/T/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)

In [17]:
class getDataLSTM_uniform:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11] 
        #ae = ['T_Mars300_sims10k_f7A11s3_c369', "conv2d_3", 1620, [12,15,9], -7] 
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)   
        
        dictToOpen = "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str([12000]) + "sims_0.9trPercent.txt"
        with open(self.pathSave + dictToOpen, "rb") as myFile:
            Dict_processed_data_monly = load(myFile)
        
        self.oMax = Dict_processed_data_monly['pMax']
        self.oMin = Dict_processed_data_monly['pMin']            
            
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = [val[2]]
                else:
                    innertimeList.append(val[2])    
                    
            simList.append(dictInd)
            timeList.append(innertimeList)
            
            indexerVec = []
            tol = 1e-3
            
            numSims = len(simList)
            maxTimeSteps = 100
            spacedTimeVec = np.linspace(0,0.04913813148788927,maxTimeSteps)
            spaced_timeList = []
            
            for ind, sim in enumerate(simList):
                innerTimeList = []
                for time in spacedTimeVec:
                    whereVec = np.where((time>=np.asarray(timeList[ind])-tol) & (time<=np.asarray(timeList[ind])+tol))[0]
                    if len(whereVec) > 0:
                        innerTimeList.append(timeList[ind][whereVec[-1]])
                    else:
                        innerTimeList.append(-1)
                spaced_timeList.append(innerTimeList)
            
            print(numSims)            
            self.x_data = np.zeros((numSims,maxTimeSteps,ae[3][0]*ae[3][1]*ae[3][2]+6))
            self.y_data = np.zeros((numSims,maxTimeSteps,ae[3][0]*ae[3][1]*ae[3][2]))  

            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            for ind, i in enumerate(simList):
                if ind%10==0:
                    print(ind)
                dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i) + ".txt"
                with open(dictToOpen, "rb") as myFile:
                    profiles = load(myFile)
                    
                for ind_t,t in enumerate(spaced_timeList[ind]):
                    if t != -1:
                        field = profiles['Dict_2D_' + sVar + '_time_' + str(t) + '_' + str(0)]
                        field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                        field = (field-self.oMin)/(self.oMax-self.oMin)      
                        field = encoder.predict(field).flatten()
                        self.x_data[ind,ind_t,6:] = field
                        if ind_t <99:
                            self.y_data[ind,ind_t+1,:] = field
                        mask = 1
                    else:
                        mask = 0
                    
                    self.x_data[ind,ind_t,0:6] = [mask, \
                            np.log10(float(profiles['InputValues'+ str(0)][1].split('=')[1])), \
                                    float(profiles['InputValues'+ str(0)][5].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][6].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][7].split('=')[1]), \
                                    float(profiles['InputValues'+ str(0)][8].split('=')[1].split("_")[0].split(".ini")[0])] 
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:,0:6],axis=(0,1))
                paraMin = np.amin(m[:,:,0:6],axis=(0,1))
                print(paraMax,paraMin)
                if s==0:
                    self.xMax = paraMax
                    self.xMin = paraMin
                else:
                    paraMax = self.xMax
                    paraMin = self.xMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,:,ind] = np.divide(m[:,:,ind]-paraMin[ind],val)
                return m        
            
            self.x_data = normalizeMatrix(self.x_data)
            
            if s==0:
                self.yMax = np.amax(self.y_data)
                self.yMin = np.amin(self.y_data)

            #print("NOT normalizing encoded representation between 0 and 1")
            self.y_data = (self.y_data-self.yMin)/(self.yMax-self.yMin) 
            self.x_data[:,:,6:] = (self.x_data[:,:,6:]-self.yMin)/(self.yMax-self.yMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin)
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.y_data[0:100,:,:].flatten(),'r.')
            plt.show()
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:,:]
            self.y_data = self.y_data[indices,:,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_data_ae840_uniform.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_cv_ae840_uniform.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/LSTM/" + sVar + "/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_test_ae840_uniform.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [18]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.98trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataLSTM_uniform(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['paraMax'] = data.xMax
            Dict_processed_data['paraMin'] = data.xMin
            Dict_processed_data['postEncodedOMax'] = data.yMax
            Dict_processed_data['postEncodedOMin'] = data.yMin

            with open(pathSave+ "processedDicts/Mars_new/LSTM/T/Dict_processed_data_MarsNew_2D_LSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_uniform.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)

In [19]:
class getDataMDN:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11]   
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)   
        
        self.oMax = indexerDict['pMax']
        self.oMin = indexerDict['pMin']
        
        
        maxTimeSteps = 0
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                    if timeSteps > maxTimeSteps:
                        maxTimeSteps = timeSteps
        
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0            
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                
            print(numSims,maxTimeSteps)
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            totalSamples = len(indexerVec)
            
            self.y_data = np.zeros((numSims,5))
            self.x_data = np.zeros((numSims,maxTimeSteps,ae[3][0]*ae[3][1]*ae[3][2]+2))
                    
            dictPrev = -1
            dictInd = -1
            for ind, i in enumerate(indexerVec):
                if i[0] != dictPrev:
                    dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i[0]) + ".txt"
                    with open(dictToOpen, "rb") as myFile:
                        profiles = load(myFile)
                    dictPrev = i[0]
                    timeStep = 0
                    dictInd += 1
                    
                    self.y_data[dictInd,:] = [np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]

                field = profiles['Dict_2D_' + sVar + '_time_' + str(i[2]) + '_' + str(i[1])]
                field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                field = (field-self.oMin)/(self.oMax-self.oMin) 
                self.x_data[dictInd,timeStep,2:] = encoder.predict(field).flatten()
                
                if ind != len(indexerVec)-1 and indexerVec[ind][0] == indexerVec[ind+1][0]:
                    t_delta = float(i[2])
                    mask = 1
                else:
                    t_delta = 0
                    mask = 0
                    self.x_data[dictInd,timeStep+1:,0:2] = [mask, t_delta]
                    self.x_data[dictInd,timeStep+1:,2:] = -1.1

                self.x_data[dictInd,timeStep,0:2] = [mask, t_delta]
                
                timeStep += 1
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:],axis=0)
                paraMin = np.amin(m[:,:],axis=0)
                if s==0:
                    self.yMax = paraMax
                    self.yMin = paraMin
                else:
                    paraMax = self.yMax
                    paraMin = self.yMin    
                paraNorms = paraMax-paraMin
                print(paraNorms)
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m        
            
            self.y_data = normalizeMatrix(self.y_data)
            
            if s==0:
                self.xMax = np.amax(self.x_data[:,:,2:])
                self.xMin = np.amin(self.x_data[:,:,2:])
                self.deltatMax = np.amax(self.x_data[:,:,1])
                self.deltatMin = np.amin(self.x_data[:,:,1])

            self.x_data[:,:,2:] = (self.x_data[:,:,2:]-self.xMin)/(self.xMax-self.xMin) 
            self.x_data[:,:,1] = (self.x_data[:,:,1]-self.deltatMin)/(self.deltatMax-self.deltatMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin, self.deltatMax, self.deltatMin)
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.y_data[0:100,:].flatten(),'r.')
            plt.show()
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:,:]
            self.y_data = self.y_data[indices,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_data.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_cv.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDN_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_x_test.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [20]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.9trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataMDN(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['paraMax'] = data.xMax
            Dict_processed_data['paraMin'] = data.xMin
            Dict_processed_data['postEncodedOMax'] = data.yMax
            Dict_processed_data['postEncodedOMin'] = data.yMin

            with open(pathSave+ "processedDicts/Mars_new/MDN/T/Dict_processed_data_MarsNew_2D_ConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)

In [21]:
class getDataMDN_convlstm:
    def __init__(self,_numSims, _pathSave, numChannels, sVar, indexerDict):
        self.pathSave = _pathSave
        self.numSims = _numSims  
        self.sVar = sVar
        self.indexerDict = indexerDict
        
        ae = ['T_Mars300_sims10k_f5A7s2_c3to24_tanh_l2reg_wlrsc', "conv2d_6", 840, [5,7,24], -11]   
        autoencoder = load_model(self.pathSave + '/TrainedNetworks/autoencoder/' + ae[0] + '.hdf5')
        autoencoder.summary()
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(ae[1]).output)   
        
        self.oMax = indexerDict['pMax']
        self.oMin = indexerDict['pMin']
        
        
        maxTimeSteps = 0
        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = 0
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    dictInd = val[0]
                    timeSteps = 1 
                else:
                    timeSteps += 1
                    if timeSteps > maxTimeSteps:
                        maxTimeSteps = timeSteps

        for s in range(3):
            if s==0:
                stringInt = "train"
            elif s==1:
                stringInt = "cv"
            else:
                stringInt = "test"
            
            indexerVec = self.indexerDict["Indexer_" + stringInt]

            indexerVec = [tuple(s) for s in indexerVec]
            dtype = [('dict', int), ('useless', int), ('time', float)]
            indexerVec = np.array(indexerVec, dtype=dtype)
            indexerVec = np.sort(indexerVec, order=['dict', 'time']) 
            indexerVec = indexerVec.tolist()
            
            numSims = -1
            simList = []
            timeList = []
            dictInd = -1
            for ind,val in enumerate(indexerVec):
                if val[0] != dictInd:
                    numSims += 1
                    if dictInd != -1:
                        timeList.append(innertimeList)
                        simList.append(dictInd)
                    dictInd = val[0]
                    innertimeList = []
                else:
                    innertimeList.append(val[2])
               
            print(numSims,maxTimeSteps)
            
            numStateVariables = numChannels #T, x, y
            sizeStateVariable = self.indexerDict["x"].shape[0] 

            numLayers = 302 #202
            sizeEachLayer = int(sizeStateVariable/numLayers)
            
            maxTimeSteps = 50
            spacedTimeVec = np.linspace(0,0.04913813148788927,maxTimeSteps)
            
            indexerVec = []
            tol = 1e-3
            
            for ind, sim in enumerate(simList):
                innerTimeList = []
                for time in spacedTimeVec:
                    whereVec = np.where((time>=np.asarray(timeList[ind])-tol) & (time<=np.asarray(timeList[ind])+tol))[0]
                    if len(whereVec) > 0:
                        indexerVec.append([sim,0,timeList[ind][whereVec[-1]]])
                    else:
                        indexerVec.append([sim,0,-1])
            
            self.y_data = np.zeros((numSims,5))
            self.x_data = np.zeros((numSims,maxTimeSteps,ae[3][0],ae[3][1],ae[3][2]))
                    
            dictPrev = -1
            dictInd = -1            
            for ind, i in enumerate(indexerVec):
                if i[0] != dictPrev:
                    dictToOpen = "/plp_user/agar_si/gaia/python/Data/2D/Mars_new/" + sVar + "/savedDict_profiles_Mars_new_2D_" \
                                                                            + sVar + "_only_Dict" + str(i[0]) + ".txt"
                    with open(dictToOpen, "rb") as myFile:
                        profiles = load(myFile)
                    
                   
                    dictPrev = i[0]
                    timeStep = 0
                    dictInd += 1
                    
                    self.y_data[dictInd,:] = [np.log10(float(profiles['InputValues'+ str(i[1])][1].split('=')[1])), \
                                float(profiles['InputValues'+ str(i[1])][5].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][6].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][7].split('=')[1]), \
                                float(profiles['InputValues'+ str(i[1])][8].split('=')[1].split("_")[0].split(".ini")[0])]

                if i[2] != -1:
                    field = profiles['Dict_2D_' + sVar + '_time_' + str(i[2]) + '_' + str(i[1])]
                    field.shape = (1,numLayers,sizeEachLayer,numStateVariables)

                    field = (field-self.oMin)/(self.oMax-self.oMin) 
                    self.x_data[dictInd,timeStep,:,:,:] = encoder.predict(field)
                else:
                    self.x_data[dictInd,timeStep,:,:,:] = -1.1
                
                timeStep += 1
            
            def normalizeMatrix(m):
                paraMax = np.amax(m[:,:],axis=0)
                paraMin = np.amin(m[:,:],axis=0)
                print(paraMin)
                if s==0:
                    self.yMax = paraMax
                    self.yMin = paraMin
                else:
                    paraMax = self.yMax
                    paraMin = self.yMin    
                paraNorms = paraMax-paraMin
                for ind,val in enumerate(paraNorms):
                    if val > 0:
                        m[:,ind] = np.divide(m[:,ind]-paraMin[ind],val)
                return m        
            
            self.y_data = normalizeMatrix(self.y_data)
            
            if s==0:
                self.xMax = np.amax(self.x_data[:,:,:,:,:])
                self.xMin = np.amin(self.x_data[:,:,:,:,:])

            self.x_data[:,:,:,:,:] = (self.x_data[:,:,:,:,:]-self.xMin)/(self.xMax-self.xMin) 
            
            print(self.xMax, self.xMin, self.yMax, self.yMin) #, self.deltatMax, self.deltatMin)
            
            plt.figure()
            plt.plot(self.x_data[0:100,:,:,:,:].flatten(),'b.')
            plt.show()
            
            plt.figure()            
            plt.plot(self.y_data[0:100,:].flatten(),'r.')
            plt.show()
            
            indices = np.arange(self.x_data.shape[0])       
            random.seed(6)
            random.shuffle(indices)
            self.x_data = self.x_data[indices,:,:,:,:]
            self.y_data = self.y_data[indices,:]
            
            if s == 0:
                print("Training data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDNConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_data.txt", "wb") as f:
                    dump([self.x_data,self.y_data] , f, protocol=4)
            if s == 1:
                print("CV data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDNConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_cv.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)
            if s == 2:
                print("Test data: " + str(self.x_data.shape) + str(self.y_data.shape))
                with open(pathSave+ "processedDicts/Mars_new/MDN/" + sVar + "/Dict_processed_data_MarsNew_2D_MDNConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840_x_test.txt", "wb") as f:
                    dump([self.x_data,self.y_data], f, protocol=4)

        self.x = self.indexerDict["x"]
        self.y = self.indexerDict["y"]
        
        profiles = {}

In [22]:
runThisCell=False
if runThisCell:
    numSims = [12000]
    numChannels = 1
    sVariables = ['T'] #, 'T']

    dictToOpen = pathSave + "processedDicts/Mars_new/T/Dict_NewMars_preProcessed_" + str(numSims) + "sims_0.9trPercent.txt"
    with open(dictToOpen, "rb") as myFile:
        indexerDict = load(myFile)

    for sVar in sVariables:
        for n in numSims:
            Dict_processed_data = {}
            data = getDataMDN_convlstm(n, pathSave, numChannels, sVar, indexerDict)
            Dict_processed_data['x'] = data.x 
            Dict_processed_data['y'] = data.y 
            Dict_processed_data['paraMax'] = data.xMax
            Dict_processed_data['paraMin'] = data.xMin
            Dict_processed_data['postEncodedOMax'] = data.yMax
            Dict_processed_data['postEncodedOMin'] = data.yMin

            with open(pathSave+ "processedDicts/Mars_new/MDN/T/Dict_processed_data_MarsNew_2D_MDNConvLSTM_" + sVar + '_' + str(numSims) + "sims_" + str(numChannels) + "channel_ae840.txt", "wb") as f:
                dump(Dict_processed_data, f, protocol=4)